In [1]:
import pandas as pd
import numpy as np
import os
import glob
import torch
from tqdm import tqdm
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
# from recbole.quick_start import load_data_and_model
from recbole.utils.case_study import full_sort_scores, full_sort_topk
from recbole.utils import init_seed, get_model, get_trainer
from recbole.data.interaction import Interaction
from recbole.evaluator import Evaluator
from recbole.evaluator.collector import DataStruct
from tqdm import tqdm
from collections import defaultdict
import time
import warnings

warnings.filterwarnings('ignore')

In [2]:
from utils.utils import (
    latest_checkpoint,
    create_ground_truth,
    calculate_recall_at_k,
    min_max_scale_excluding_inf,
    get_top_k_indices
)

In [3]:
def load_data_and_model(model_file):

    checkpoint = torch.load(model_file)
    config = checkpoint["config"]
    init_seed(config["seed"], config["reproducibility"])

    dataset = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)

    init_seed(config["seed"], config["reproducibility"])
    model = get_model(config["model"])(config, train_data._dataset).to(config["device"])
    model.load_state_dict(checkpoint["state_dict"])
    model.load_other_parameter(checkpoint.get("other_parameter"))

    return config, model, dataset, train_data, valid_data, test_data

In [4]:
model_info_list = [
    # General
    {'model': 'EASE', 'model_type': 'general'},
    {'model': 'ADMMSLIM', 'model_type': 'general'},
    {'model': 'CDAE', 'model_type': 'general'},
    # {'model': 'SLIMElastic', 'model_type': 'general'},
    {'model': 'RecVAE', 'model_type': 'general'},
    {'model': 'MultiVAE', 'model_type': 'general'},
    {'model': 'MultiDAE', 'model_type': 'general'},
    {'model': 'LightGCN', 'model_type': 'general'},
    
    # Context
    {'model': 'DCNV2', 'model_type': 'context'},
    {'model': 'DeepFM', 'model_type': 'context'},

    # Sequential
    {'model': 'BERT4Rec', 'model_type': 'sequential'},
    {'model': 'GRU4Rec', 'model_type': 'sequential'},
    {'model': 'GRU4RecF', 'model_type': 'sequential'},
    {'model': 'SASRec', 'model_type': 'sequential'},
    # {'model': 'SASRecF', 'model_type': 'sequential'},
]

In [5]:
configs = []
checkpoints = []
for i in range(len(model_info_list)):
    model_name = model_info_list[i]['model']
    checkpoint_dir = f'model/saved/{model_name}'
    checkpoint_pattern = os.path.join(checkpoint_dir, f"{model_name}-*.pth")
    checkpoint_files = glob.glob(checkpoint_pattern)
    # 최신 체크포인트 파일 선택
    checkpoint_path = max(checkpoint_files, key=os.path.getmtime)
    print(f"{model_name}: {checkpoint_path}")
    checkpoints.append(checkpoint_path)

EASE: model/saved/EASE/EASE-Nov-26-2024_10-46-47.pth
ADMMSLIM: model/saved/ADMMSLIM/ADMMSLIM-Nov-26-2024_15-27-37.pth
CDAE: model/saved/CDAE/CDAE-Nov-26-2024_12-29-41.pth
RecVAE: model/saved/RecVAE/RecVAE-Nov-26-2024_12-41-37.pth
MultiVAE: model/saved/MultiVAE/MultiVAE-Nov-25-2024_17-32-14.pth
MultiDAE: model/saved/MultiDAE/MultiDAE-Nov-26-2024_15-01-39.pth
LightGCN: model/saved/LightGCN/LightGCN-Nov-26-2024_01-56-18.pth
DCNV2: model/saved/DCNV2/DCNV2-Nov-25-2024_22-44-55.pth
DeepFM: model/saved/DeepFM/DeepFM-Nov-25-2024_20-57-38.pth
BERT4Rec: model/saved/BERT4Rec/BERT4Rec-Nov-27-2024_23-39-59.pth
GRU4Rec: model/saved/GRU4Rec/GRU4Rec-Nov-27-2024_14-03-49.pth
GRU4RecF: model/saved/GRU4RecF/GRU4RecF-Nov-28-2024_01-07-18.pth
SASRec: model/saved/SASRec/SASRec-Nov-27-2024_19-27-55.pth


#### 오류 발생 시:

- `numpy` 최신 버전을 사용할 경우, `np.long` 자료형이 더 이상 지원되지 않으므로, `numpy` 버전을 낮추거나 RecBole에서 `np.long`을 사용하는 부분을 수정해야 함
- 여기서는 RecBole의 `/lib/python3.11/site-packages/recbole/model/layers.py` 부분과, `lib/python3.11/site-packages/recbole/model/abstract_recommender.py`에 등장하는 `np.long`을 `np.int64`로 수정하여 오류를 해결함

In [6]:
configs = []
models = []
datasets = []
train_datasets = []
valid_datasets = []

for i in range(len(model_info_list)):
    start = time.time()
    config, model, dataset, train_data, valid_data, _ = load_data_and_model(checkpoints[i])
    print(f"Loading {model_info_list[i]['model']}: {time.time()-start:.4f}s")
    configs.append(config)
    models.append(model)
    datasets.append(dataset)
    train_datasets.append(train_data)
    valid_datasets.append(valid_data)

Loading EASE: 63.2947s
Loading ADMMSLIM: 590.0921s


Max value of user's history interaction records has reached 42.34723854289072% of the total.


Loading CDAE: 63.7542s


Max value of user's history interaction records has reached 42.34723854289072% of the total.


Loading RecVAE: 76.2795s


Max value of user's history interaction records has reached 42.34723854289072% of the total.


Loading MultiVAE: 76.3642s


Max value of user's history interaction records has reached 42.34723854289072% of the total.


Loading MultiDAE: 79.5287s
Loading LightGCN: 81.5287s
Loading DCNV2: 76.4887s
Loading DeepFM: 85.3699s


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [ ]:
# temp_save_files = torch.load('model/temp1_saved_files.pth')

In [ ]:
# configs = temp_save_files2['configs']
# models = temp_save_files2['models']
# datasets = temp_save_files2['datasets']
# train_datasets = temp_save_files2['train_datasets']
# valid_datasets = temp_save_files2['valid_datasets']

In [11]:
for i in range(9, len(model_info_list)):
    start = time.time()
    config, model, dataset, train_data, valid_data, _ = load_data_and_model(checkpoints[i])
    print(f"Loading {model_info_list[i]['model']}: {time.time()-start:.4f}s")
    configs.append(config)
    models.append(model)
    datasets.append(dataset)
    train_datasets.append(train_data)
    valid_datasets.append(valid_data)

Loading BERT4Rec: 209.8402s
Loading GRU4Rec: 211.2989s
Loading GRU4RecF: 216.2147s
Loading SASRec: 201.8457s


#### 중간 저장

- `load_data_and_model`이 시간이 오래 걸림 
    - EASE, ADMMSLIM 등 `TRADITIONAL` 모델의 경우 `__init__` 메서드에서 학습을 하기 때문
    - 그 외에도 `DataLoader`를 각 모델마다 불러오는 과정이 시간을 꽤나 잡아먹음
- 이 과정을 한 번만 수행한 다음, 모두 딕셔너리로 만들어 `.pth` 파일로 저장하는 방식을 취함

In [12]:
temp_save_files = {
    'configs': configs,
    'models': models,
    'datasets': datasets,
    'train_datasets': train_datasets,
    'valid_datasets': valid_datasets
}
save_path = 'model/temp_saved_files.pth'
torch.save(temp_save_files, save_path, pickle_protocol=4)

In [13]:
import gc
gc.collect()

0

In [14]:
sample_submission = pd.read_csv(os.path.join(config['eval_path'], 'sample_submission.csv'))
sample_submission.columns = ['user_id', 'item_id']
test_users = sample_submission['user_id'].unique().tolist()
test_users = [str(user) for user in test_users]

model_recommendations = []
for full_dataset, valid_data, best_model in tqdm(zip(datasets, valid_datasets, models), total=len(models)):

    uid_series = full_dataset.token2id(full_dataset.uid_field, test_users)

    batch_size = 16
    recommended_df = pd.DataFrame(columns=['user', 'item'])
    for i in tqdm(range(0, len(uid_series), batch_size)):
        batch_indices = uid_series[i:i+batch_size]
        batch_users = test_users[i:i+batch_size]

        topk_iid_list_batch = full_sort_topk(batch_indices, best_model, valid_data, k=20, device=config['device'])
        last_topk_iid_list = topk_iid_list_batch.indices
        recommended_item_list = full_dataset.id2token(full_dataset.iid_field, last_topk_iid_list.cpu()).tolist()
        temp_df = pd.DataFrame({'user': batch_users, 'item': recommended_item_list})
        recommended_df = pd.concat([recommended_df, temp_df], ignore_index=True)

    recommended_df = recommended_df.explode('item').reset_index(drop=True)
    model_recommendations.append(recommended_df)

100%|██████████| 13/13 [07:03<00:00, 32.57s/it]


#### Inference 결과 저장

- Hard Voting에 사용할 후보 모델들의 inference 결과를 모두 `.pth` 파일로 저장
- Hard Voting에서는 이 후보 inference를 가지고 수행하면 됨

In [16]:
temp_file = {
    'model_recommendations': model_recommendations
}
save_path = 'data/output/Ensemble/Ensemble_Candidates.pth'
torch.save(temp_file, save_path, pickle_protocol=4)

# output별 유사도 측정

In [ ]:
from itertools import combinations

def count_similiarity(df1, df2):
    df = pd.concat([df1,df2])
    df = df.groupby(['user','item']).size().reset_index(name='counts')
    df = df[df['counts'] > 1]
    return len(df) / len(df1) * 100

index_combinations = list(combinations(range(len(model_recommendations)), 2))

similarity_matrix = pd.DataFrame(
    np.zeros((len(models), len(models))), 
    index=[model['model'] for model in model_info_list], 
    columns=[model['model'] for model in model_info_list]
)

for i, j in index_combinations:
    df1 = model_recommendations[i]
    df2 = model_recommendations[j]
    sim = count_similiarity(df1, df2)
    df1_name = model_info_list[i]['model']
    df2_name = model_info_list[j]['model']
    # print(f'{df1_name} and {df2_name}: {sim:.2f}% similarity')
    
    similarity_matrix.loc[df1_name, df2_name] = sim
    similarity_matrix.loc[df2_name, df1_name] = sim

In [38]:
pd.options.display.float_format = '{:.2f}'.format

In [39]:
display(similarity_matrix)

,EASE,ADMMSLIM,CDAE,RecVAE,MultiVAE,MultiDAE,LightGCN,DCNV2,DeepFM,BERT4Rec,GRU4Rec,GRU4RecF,SASRec
EASE,0.00,74.06,59.21,55.39,52.00,51.32,48.58,53.23,45.19,7.53,8.15,8.27,7.15
ADMMSLIM,74.06,0.00,51.67,49.84,46.07,45.38,43.85,49.40,41.05,7.25,7.77,7.96,6.93
CDAE,59.21,51.67,0.00,60.72,57.44,58.23,56.54,54.32,52.13,7.93,8.82,8.81,7.94
RecVAE,55.39,49.84,60.72,0.00,59.65,58.55,62.44,65.09,58.67,7.78,8.57,8.60,7.64
MultiVAE,52.00,46.07,57.44,59.65,0.00,59.12,53.18,54.72,49.47,7.67,8.32,8.33,7.47
MultiDAE,51.32,45.38,58.23,58.55,59.12,0.00,53.18,51.55,48.97,7.72,8.37,8.38,7.64
LightGCN,48.58,43.85,56.54,62.44,53.18,53.18,0.00,57.89,58.03,7.49,8.30,8.27,7.66
DCNV2,53.23,49.40,54.32,65.09,54.72,51.55,57.89,0.00,56.24,7.90,8.80,8.70,7.61
DeepFM,45.19,41.05,52.13,58.67,49.47,48.97,58.03,56.24,0.00,7.35,8.04,8.07,7.23
BERT4Rec,7.53,7.25,7.93,7.78,7.67,7.72,7.49,7.90,7.35,0.00,51.78,47.29,40.90


# 모델 가중치 + 랭킹 가중치

In [154]:
weights = [
    # General
    {'model': 'EASE',     'weight': 1},     #0   0.2366
    {'model': 'ADMMSLIM', 'weight': 1},     #1   0.2309
    {'model': 'CDAE',     'weight': 1},     #2   0.2203
    {'model': 'RecVAE',   'weight': 1},     #3   0.2048
    # {'model': 'MultiVAE', 'weight': 1},     #4   0.2016
    # {'model': 'MultiDAE', 'weight': 1},     #5   0.2013
    # {'model': 'LightGCN', 'weight': 1},     #6   0.1896
    
    # Context
    # {'model': 'DCNV2',    'weight': 1},     #7   0.1937
    # {'model': 'DeepFM',   'weight': 1},     #8   0.1785

    # Sequential
    {'model': 'BERT4Rec', 'weight': 0.3},     #9   0.1847
    {'model': 'GRU4Rec',  'weight': 0.3},     #10  0.1437
    {'model': 'GRU4RecF', 'weight': 0.3},     #11  0.1567
    # {'model': 'SASRec',   'weight': 1},     #12  0.1161
]

In [100]:
for i in range(len(model_recommendations)):
    model_recommendations[i]['user'] = model_recommendations[i]['user'].astype(int)
    model_recommendations[i]['item'] = model_recommendations[i]['item'].astype(int)

In [155]:
# rec_results = model_recommendations[0:4] + model_recommendations[9:11]
rec_results = model_recommendations[0:4] + model_recommendations[9:12]

In [156]:
# 추천 결과를 저장할 딕셔너리 {user_id: {item_id: weighted_score}}
weighted_recommendations = defaultdict(lambda: defaultdict(float))

# 가중 합 계산 (Rank Aggregation 반영)
print("Calculating weighted recommendations with rank aggregation...")
for model_idx, df in tqdm(enumerate(rec_results), total=len(rec_results)):
    weight = weights[model_idx]['weight']
    grouped = df.groupby("user")
    
    # 각 사용자별로 순위를 기반으로 점수 계산
    for user, group in grouped:
        # 아이템별 순위를 계산
        group["rank"] = range(1, len(group) + 1)
        
        # 순위를 점수에 반영 (역순위 사용: 높은 순위 -> 높은 점수)
        for _, row in group.iterrows():
            item = row["item"]
            rank = row["rank"]
            # 점수 계산: 모델 가중치 + 역순위 가중치
            rank_weight = 1 / (rank+60)  # 예: 1등 = 1, 2등 = 0.5, ...
            weighted_recommendations[user][item] += weight * rank_weight

# top@10 계산 및 DataFrame 생성
top_k = 10
recommendation_list = []

print("\nGenerating top@10 recommendations for each user...")
for user, item_scores in tqdm(weighted_recommendations.items(), total=len(weighted_recommendations)):
    # 아이템별 점수를 기준으로 정렬하여 top@10 추출
    sorted_items = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
    for item, _ in sorted_items:
        recommendation_list.append({"user": user, "item": item})

# 최종 추천 결과 DataFrame
final_recommendations_df = pd.DataFrame(recommendation_list)

# 결과 출력
print("\nFinal recommendations:")
print(final_recommendations_df)

Calculating weighted recommendations with rank aggregation...


100%|██████████| 7/7 [04:37<00:00, 39.68s/it]



Generating top@10 recommendations for each user...


100%|██████████| 31360/31360 [00:00<00:00, 42245.32it/s]



Final recommendations:
          user   item
0           11   4886
1           11   8961
2           11   4370
3           11      2
4           11  32587
...        ...    ...
313595  138493    110
313596  138493   2762
313597  138493  32587
313598  138493  48394
313599  138493   2174

[313600 rows x 2 columns]


In [157]:
final_recommendations_df.head(20)

,user,item
0,11,4886
1,11,8961
2,11,4370
3,11,2
4,11,32587
5,11,37386
6,11,40815
7,11,7373
8,11,7438
9,11,8861


In [158]:
final_recommendations_df.to_csv("./data/output/Ensemble/output_ensemble9.csv", index=False)

In [159]:
df1 = pd.read_csv("./data/output/Ensemble/output_ensemble1.csv")
df2 = pd.read_csv("./data/output/Ensemble/output_ensemble9.csv")
count_similiarity(df1, df2)

93.74298469387755

# General: Sequential = 7:3

In [139]:
df_general = pd.read_csv("./data/output/Ensemble/output_ensemble7_general.csv")
df_sequential = pd.read_csv("./data/output/Ensemble/output_ensemble7_sequential.csv")

In [142]:
df_general

,user,item
0,11,4886
1,11,4370
2,11,8961
3,11,2
4,11,40815
...,...,...
313595,138493,48394
313596,138493,8961
313597,138493,5349
313598,138493,110


In [150]:
# 1. CSV 파일 읽기
a_csv = pd.read_csv("./data/output/Ensemble/output_ensemble7_general.csv")
b_csv = pd.read_csv("./data/output/Ensemble/output_ensemble7_sequential.csv")

# 결과 저장용 리스트
result = []

# 유저 리스트 생성 (a와 b에서 공통 유저)
users = a_csv['user'].unique()

# 2. 유저별로 아이템 추출
for user in tqdm(users):
    # a.csv에서 해당 유저의 상위 5개 추출
    a_items = a_csv[a_csv['user'] == user]['item'].values[:9]
    
    # b.csv에서 해당 유저의 아이템 중 a_items를 제외한 상위 5개 추출
    b_items = b_csv[b_csv['user'] == user]['item'].values
    b_items = [item for item in b_items if item not in a_items][:1]
    
    # 최종 10개 아이템 (a_items + b_items)
    combined_items = list(a_items) + b_items
    
    # 결과 저장 (user, item) 형식
    for item in combined_items:
        result.append({'user': user, 'item': item})

# 3. 결과를 DataFrame으로 변환
final_df = pd.DataFrame(result)

# 4. 새로운 CSV 파일로 저장
# final_df.to_csv('final_combined.csv', index=False)

100%|██████████| 31360/31360 [00:40<00:00, 779.28it/s]


In [151]:
final_df.to_csv('data/output/Ensemble/ouput_ensemble8.csv', index=False)

In [152]:
final_df.head(20)

,user,item
0,11,4886
1,11,4370
2,11,8961
3,11,2
4,11,40815
5,11,32587
6,11,7373
7,11,47
8,11,37386
9,11,5952


In [153]:
df1 = pd.read_csv("./data/output/Ensemble/output_ensemble1.csv")
df2 = pd.read_csv("./data/output/Ensemble/ouput_ensemble8.csv")
count_similiarity(df1, df2)

78.56983418367346